In [ ]:
import numpy as np
import pandas as pd
import json

def trajectory(a_x, a_y, a_z, M_leg, M_ball, t, e, ip=0.01):
    # calculating v_leg in x, y, z
    v_l_x = a_x * ip
    v_l_y = a_y * ip
    v_l_z = a_z * ip

    # calculate v_ball in x, y, z
    v_b_x = v_l_x * (M_leg / (M_leg + M_ball)) * (1 + e)
    v_b_y = v_l_y * (M_leg / (M_leg + M_ball)) * (1 + e)
    v_b_z = v_l_z * (M_leg / (M_leg + M_ball)) * (1 + e)

    # Calculate the trajectory of the ball.
    x_t = v_b_x * t
    y_t = v_b_y * t
    z_t = max(v_b_z * t - 0.5 * (9.81) * t ** 2, 0)  # Ensure z_t is not negative

    return [x_t, y_t, z_t]

def generate_realistic_trajectory_data(num_samples, t_max, num_time_steps):
    """Generate realistic trajectory data."""
    # realistic acceleration ranges
    ax = np.random.uniform(0, 2000, num_samples)  # horizontal acceleration
    ay = np.random.uniform(0, 2000, num_samples)  # horizontal acceleration
    az = np.random.uniform(0, 1500, num_samples)  # vertical acceleration, including gravity

    t = np.linspace(0, t_max, num_time_steps)  # Time intervals

    data = []
    penalty_kick_dist = 11
    actual_goal_height = 2.4
    actual_goal_length = 7.3
    actual_goal_width = 3.0
    goal_W_bound = penalty_kick_dist + actual_goal_width
    goal_L_bound = actual_goal_length/2;
    goal_H_bound = actual_goal_height;

    for i in range(num_samples):
        scored = False
        last_x, last_y, last_z = 0, 0, 0  # default start position
        for time in t:
            if not scored:
                x_t, y_t, z_t = trajectory(ax[i], ay[i], az[i], 9, 0.3, time, 0.7, 0.01)
                in_goal = (z_t >= 0) and (z_t <= goal_H_bound) and (y_t >= -goal_L_bound) and (y_t <= goal_L_bound) and (x_t >= penalty_kick_dist) and (x_t <= goal_W_bound)
                if in_goal:
                    scored = True
                    last_x, last_y, last_z = x_t, y_t, z_t
            else:
                x_t, y_t, z_t = last_x, last_y, last_z  # maintain last position

            data.append({'ax': ax[i], 'ay': ay[i], 'az': az[i], 'time': time, 'x': x_t, 'y': y_t, 'z': z_t, 'in_goal': in_goal})

    return data

def convert_bool(data):
    for d in data:
        d['in_goal'] = bool(d['in_goal'])
    return data

# generating the data
num_samples = 100
t_max = 5.0
num_time_steps = 13

data = generate_realistic_trajectory_data(num_samples, t_max, num_time_steps)
data = convert_bool(data)  # convert NumPy boolean to Python boolean

# saving the data to a JSON file
file_path = 'realistic_trajectory_data.json'
with open(file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)

print(f"Data has been successfully saved to {file_path}.")


Data has been successfully saved to realistic_trajectory_data.json.
